In [6]:
%matplotlib inline
import lightgbm as lgb
from sklearn import model_selection
from sklearn.metrics import *
import datetime
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('data/train_final4.csv')
test = pd.read_csv('data/test_final3.csv')

# transforming dtype

In [7]:
train['transaction_date'] = pd.to_datetime(train['transaction_date'], format='%Y%m%d')
train['membership_expire_date'] = pd.to_datetime(train['membership_expire_date'], format='%Y%m%d')
train['registration_init_time'] = pd.to_datetime(train['registration_init_time'], format='%Y%m%d')

trans_date = train['transaction_date']
memb_date = train['membership_expire_date']
regi_date = train['registration_init_time']

ls = []
ls2 = []
ls3 = []

for i in range(len(trans_date)):
    ls.append(trans_date[i].toordinal())
    ls2.append(memb_date[i].toordinal())
    ls3.append(regi_date[i].toordinal())

train['transaction_date'] = ls
train['membership_expire_date'] = ls2
train['registration_init_time'] = ls3

In [8]:
train['is_churn'] = train['is_churn'].astype('category')
train['payment_method_id'] = train['payment_method_id'].astype('category')
train['payment_plan_days'] = train['payment_plan_days'].astype('category')
train['is_auto_renew'] = train['is_auto_renew'].astype('category')
train['is_cancel'] = train['is_cancel'].astype('category')
train['city'] = train['city'].astype('category')
train['gender'] = train['gender'].astype('category')
train['registered_via'] = train['registered_via'].astype('category')
train['age'] = train['age'].astype('int')
train['disc_user'] = train['disc_user'].astype('category')
# train['transaction_date'] = train['transaction_date'].astype('int')
# train['membership_expire_date'] = train['membership_expire_date'].astype('int')
# train['registration_init_time'] = train['registration_init_time'].astype('int')

In [9]:
a = test['transaction_date'].values
a[32648] = 20000101
a = pd.to_datetime(a, format='%Y%m%d', box=True)
b = test['membership_expire_date'].values
b[32648] = 20000201
b = pd.to_datetime(b, format='%Y%m%d')
# a_ls = []
# for i in a:
#     a_ls.append(a.strftime('%Y%m%d'))

test['transaction_date'] = pd.to_datetime(test['transaction_date'], format='%Y%m%d')
test['membership_expire_date'] = pd.to_datetime(test['membership_expire_date'], format='%Y%m%d')
test['registration_init_time'] = pd.to_datetime(test['registration_init_time'], format='%Y%m%d')

trans_date2 = test['transaction_date']
memb_date2 = test['membership_expire_date']
regi_date2 = test['registration_init_time']

ls_ = []
ls_2 = []
ls_3 = []

for i in range(len(trans_date2)):
    ls_.append(trans_date2[i].toordinal())
    ls_2.append(memb_date2[i].toordinal())
    ls_3.append(regi_date2[i].toordinal())

test['transaction_date'] = ls_
test['membership_expire_date'] = ls_2
test['registration_init_time'] = ls_3

test.set_value(index=32648, col='transaction_date',value=0)
test.set_value(index=32648, col='membership_expire_date', value=0)

,msno,is_churn,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,num_unq,...,gender,registered_via,registration_init_time,age,pro_25,pro_50,pro_75,pro_985,pro_100,disc_user
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,41.0,30.0,99.0,1.0,736406,736437,0.0,515.732526,...,0,7.0,735797,29,0.153466,0.036997,0.023353,0.026450,0.759734,0.0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,34.0,30.0,149.0,1.0,736419,736449,0.0,2273.000000,...,0,9.0,732249,28,0.204994,0.065598,0.046590,0.048826,0.633992,0.0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,41.0,30.0,99.0,1.0,736403,736434,0.0,149.000000,...,0,7.0,735538,34,0.427027,0.102703,0.005405,0.010811,0.454054,0.0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,41.0,30.0,99.0,1.0,736415,736446,0.0,515.732526,...,0,9.0,735247,20,0.153466,0.036997,0.023353,0.026450,0.759734,0.0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,30.0,30.0,129.0,1.0,736410,736440,0.0,515.732526,...,1,13.0,736320,36,0.153466,0.036997,0.023353,0.026450,0.759734,0.0
5,nu1jZ/flvSaXRg0U9Es+xmo2KlAXq/q+mhcWFStwm9w=,0,36.0,30.0,180.0,0.0,736414,736444,0.0,2067.000000,...,0,4.0,735996,29,0.050806,0.016532,0.010484,0.012097,0.910081,0.0
6,biDTtgK83fEWXJDaxzD1eh22dion/h3odeIbS7qJMmY=,0,37.0,30.0,149.0,1.0,736393,736423,0.0,156.000000,...,0,3.0,734933,26,0.049261,0.049261,0.029557,0.019704,0.852217,0.0
7,k/8uwi/iM9LZmRAIWXLqpZY6ENomXAscwsQsh6PxcTw=,0,41.0,30.0,99.0,1.0,736419,736449,0.0,515.732526,...,1,9.0,735251,34,0.153466,0.036997,0.023353,0.026450,0.759734,0.0
8,18rQ/746SjA6nBx325UsyhfsDhu4tK01FXFxHWZjw20=,0,40.0,30.0,149.0,1.0,736396,736427,0.0,971.000000,...,0,9.0,735030,32,0.016716,0.005900,0.007866,0.005900,0.963618,0.0
9,2V13OCoWx6vqKr/ZzNmKFrmnC2FtR4SWMz5C5Hi02PY=,0,40.0,30.0,149.0,1.0,736396,736426,0.0,18.000000,...,0,4.0,736086,32,0.000000,0.000000,0.000000,0.000000,1.000000,0.0


In [10]:
test['is_churn'] = test['is_churn'].astype('category')
test['payment_method_id'] = test['payment_method_id'].astype('category')
test['payment_plan_days'] = test['payment_plan_days'].astype('category')
test['is_auto_renew'] = test['is_auto_renew'].astype('category')
test['is_cancel'] = test['is_cancel'].astype('category')
test['city'] = test['city'].astype('category')
test['gender'] = test['gender'].astype('category')
test['registered_via'] = test['registered_via'].astype('category')
test['age'] = test['age'].astype('int')
test['disc_user'] = test['disc_user'].astype('category')
# test['transaction_date'] = test['transaction_date'].astype('int')
# test['membership_expire_date'] = test['membership_expire_date'].astype('int')
# test['registration_init_time'] = test['registration_init_time'].astype('int')

In [11]:
train.to_csv('train_final_ordinal', index=False)
test.to_csv('test_final_ordinal', index=False)